# Imports

In [1]:
import classification as mt

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # adapt plots for retina displays

/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)


# Load model, dataset and brain area masks

In [2]:
!ls

APTOS2019-TAME	   misc		    README.md	      train_tame.ipynb
classification.py  models_and_misc  requirements.txt  utils
defaults	   params.json	    self_supervised   wandb
docker_image	   __pycache__	    _torchshow


In [3]:
net, train_loader, num_classes = mt.get_jupyter_items()

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


In [4]:
# Important: Set model to eval before using any interpretation methods
net.eval()

Classifier(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (norm): Layer

# Build TAME model (from scratch)

In [7]:
from tame.utilities.pl_module import TAMELIT

version = "TAME"
model = net.cpu()
layers = [
    "backbone.blocks.9",
    "backbone.blocks.10",
    "backbone.blocks.11",
]
epochs = 8

pl_model = TAMELIT(
    model_name="vit_b_16",
    model=model,
    layers=layers,
    attention_version=version,
    train_method="new",
    lr=0.001,
    epochs=epochs,
    num_classes=num_classes,
    eval_length="long",
    LeRF=True
)


In [6]:
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger(project="APTOS2019-TAME", name="random initialization")
trainer = pl.Trainer(
    enable_checkpointing=True,
    max_epochs=epochs,
    logger=wandb_logger,
    log_every_n_steps=1,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
import torch

torch.set_float32_matmul_precision("medium")

## Train

In [8]:
print(next(iter(train_loader))[0][0].shape)

torch.Size([3, 224, 224])


In [9]:
trainer.fit(pl_model, train_loader)

/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marios1861. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name           | Type               | Params
------------------------------------------------------
0 | generic        | Generic            | 22.1 M
1 | train_accuracy | MulticlassAccuracy | 0     
2 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
451 K     Trainable params
21.7 M    Non-trainable params
22.1 M    Total params
88.477    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.


## Evaluate with AD and IC metrics

In [10]:
_, test_loader, _ = mt.get_jupyter_items(train=False)

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


In [11]:
# trainer.logger = CSVLogger("logs", name="3D-TAME")
trainer.test(pl_model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

[{}]

In [12]:
wandb_logger.finalize("success")
wandb_logger.experiment.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/Accuracy,█▁▁▂▃▂▁▂▂▂▁▃▂▃▂▁▁▂▂▂▂▂▃▂▂▂▂▂▂▂▂▃▂▂▂▁▂▃▂▂
training/CE,▁███████████████████████████████████████
training/Loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/Mean,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/Var,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,7
trainer/global_step,319
training/Accuracy,0.17188
training/CE,2.67017


## Train deit-small TAME on ImageNet

In [1]:
%cd ..

/home/ntrougkas/Documents/3D-TAME


In [2]:
import os
from pathlib import Path

import timm
from dotenv import load_dotenv

from tame.utilities.pl_module import TAMELIT, LightnightDataset

load_dotenv("/home/ntrougkas/Documents/3D-TAME/.env")

epochs = 8
old_model = TAMELIT(
    model=timm.create_model("deit_small_patch16_224", pretrained=True),
    model_name="vit_b_16",
    layers=["blocks.9", "blocks.10", "blocks.11"],
    attention_version="TAME",
    normalized_data=True,
    lr=0.001,
    epochs=epochs,
    eval_length="short",
)

imagenet_data = LightnightDataset(
    dataset_path=Path(os.getenv("DATA", "./")),
    datalist_path=Path(os.getenv("LIST", "./")),
    model="vit_b_16",
    batch_size=32,
)

INFO:timm.models.helpers:Loading pretrained weights from url (https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth)
INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpo8zpjbme
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpo8zpjbme/_remote_module_non_scriptable.py


In [3]:
import torch

torch.set_float32_matmul_precision("medium")

In [4]:
from lightning.pytorch.loggers import WandbLogger
import lightning.pytorch as pl

wandb_logger = WandbLogger(project="T-TAME-small")
trainer = pl.Trainer(enable_checkpointing=True, max_epochs=epochs, logger=wandb_logger, precision=16)
trainer.fit(old_model, imagenet_data)
trainer.test(old_model, imagenet_data)

/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/lightning/fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marios1861. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name           | Type               | Params
------------------------------------------------------
0 | generic        | Generic            | 23.6 M
1 | train_accuracy | MulticlassAccuracy | 0     
2 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
1.6 M     Trainable params
22.1 M    Non-trainable params
23.6 M    Total params
94.598    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          AD 100%          │     21.06305503845215     │
│          AD 15%           │     79.13764953613281     │
│          AD 50%           │     33.58638381958008     │
│          IC 100%          │    14.050000190734863     │
│          IC 15%           │           4.25            │
│          IC 50%           │           17.25           │
└───────────────────────────┴───────────────────────────┘

[{'AD 100%': 21.06305503845215,
  'IC 100%': 14.050000190734863,
  'AD 50%': 33.58638381958008,
  'IC 50%': 17.25,
  'AD 15%': 79.13764953613281,
  'IC 15%': 4.25}]

In [5]:
import wandb

wandb.alert(
    title="Training finished",
    text="Training on deit-small finished, can finish medical transformers task",
)
wandb_logger.finalize("success")
wandb_logger.experiment.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


AD 100%,▁
AD 15%,▁
AD 50%,▁
IC 100%,▁
IC 15%,▁
IC 50%,▁
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
training/Accuracy,▃▅▆▃▃▅▅▅▁▅█▅█▇▄▅▇▇▇▅▇▆▇█▆▆▃▆▇▃▆▇▃▆▅▇▇▅█▇
training/CE,█▇▃▆▄▄▄▃▆▄▂▃▂▃▃▃▂▃▂▄▃▃▂▁▃▂▄▃▂▄▂▂▄▃▃▂▂▃▂▁
training/Loss,█▆▃▄▃▃▃▂▄▃▂▂▂▂▂▂▂▂▂▃▂▂▁▁▂▂▃▂▁▂▁▂▂▂▂▁▂▂▂▁


Training complete, now we can finetune the trained TAME model on APTOS

# Finetune TAME on APTOS

In [1]:
!ls

APTOS2019-TAME	   misc		    README.md	      train_tame.ipynb
classification.py  models_and_misc  requirements.txt  utils
defaults	   params.json	    self_supervised   wandb
docker_image	   __pycache__	    _torchshow


In [2]:
%cd medical_transformers

[Errno 2] No such file or directory: 'medical_transformers'
/home/ntrougkas/Documents/3D-TAME/medical_transformers


In [1]:
import classification as mt

net, train_loader, num_classes = mt.get_jupyter_items()
net.eval()
from tame.utilities.pl_module import TAMELIT

version = "TAME"
model = net.cpu()
layers = [
    "backbone.blocks.9",
    "backbone.blocks.10",
    "backbone.blocks.11",
]
epochs = 8

pl_model = TAMELIT(
    model_name="vit_b_16",
    model=model,
    layers=layers,
    attention_version=version,
    train_method="new",
    lr=0.0001,
    epochs=epochs,
    num_classes=num_classes,
    eval_length="long",
    LeRF=True
)

/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)


Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpah7kkekr
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpah7kkekr/_remote_module_non_scriptable.py


Use new fusion module

In [2]:
import timm

pretrained_model = TAMELIT.load_from_checkpoint(
    "/home/ntrougkas/Documents/3D-TAME/T-TAME-small/vkep09vs/checkpoints/epoch=7-step=320296.ckpt",
    model=timm.create_model("deit_small_patch16_224", pretrained=True),
)
pretrained_model.generic.attn_mech[1].fuser = pl_model.generic.attn_mech[1].fuser
pl_model.generic.attn_mech[1] = pretrained_model.generic.attn_mech[1]

INFO:timm.models.helpers:Loading pretrained weights from url (https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth)


Train

In [3]:
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
import torch

torch.set_float32_matmul_precision("medium")
wandb_logger = WandbLogger(project="APTOS2019-TAME", name="pretrained-new")
trainer = pl.Trainer(
    enable_checkpointing=True,
    precision="16-mixed",
    gradient_clip_algorithm="norm",
    max_epochs=epochs,
    logger=wandb_logger,
    log_every_n_steps=1,
)
trainer.fit(pl_model, train_loader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marios1861. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name           | Type               | Params
------------------------------------------------------
0 | generic        | Generic            | 22.1 M
1 | train_accuracy | MulticlassAccuracy | 0     
2 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
451 K     Trainable params
21.7 M    Non-trainable params
22.1 M    Total params
88.477    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
`Trainer.fit` stopped: `max_epochs=8` reached.


Test

for evaluating from checkpoint

In [34]:

net, test_loader, _ = mt.get_jupyter_items(train=False)
net.eval()
net.cpu()

pl_model = TAMELIT.load_from_checkpoint(
    # "/home/ntrougkas/Documents/3D-TAME/medical_transformers/APTOS2019-TAME/6i01rwf8/checkpoints/epoch=7-step=320.ckpt",
    "/home/ntrougkas/Documents/3D-TAME/medical_transformers/APTOS2019-TAME/d1el6nks/checkpoints/epoch=7-step=320.ckpt",
    model=net,
    eval_length="short"
)

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


In [4]:
_, test_loader, _ = mt.get_jupyter_items(train=False)
trainer.test(pl_model, test_loader)

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

[{}]

In [5]:
wandb_logger.finalize("success")
wandb_logger.experiment.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/Accuracy,█▇▄▁▁▂▂▂▂▃▂▃▂▂▁▂▂▂▂▂▂▂▁▂▃▂▂▃▃▃▂▂▁▂▃▂▂▁▂▂
training/CE,▁▄▇█████████████████████████████████████
training/Loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/Mean,█▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/Var,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,7
trainer/global_step,319
training/Accuracy,0.17188
training/CE,2.6724


## Do the same with new fuser module, retaining the older one and adapting it to new output classes

In [1]:
import classification as mt

net, train_loader, num_classes = mt.get_jupyter_items()
net.eval()
from tame.utilities.pl_module import TAMELIT

version = "TAME"
model = net.cpu()
layers = [
    "backbone.blocks.9",
    "backbone.blocks.10",
    "backbone.blocks.11",
]
epochs = 8

pl_model = TAMELIT(
    model_name="vit_b_16",
    model=model,
    layers=layers,
    attention_version=version,
    train_method="new",
    lr=0.0001,
    epochs=epochs,
    num_classes=num_classes,
    eval_length="long",
)

/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)


Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpyfbr9unm
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpyfbr9unm/_remote_module_non_scriptable.py


Adapt existing fusion module

In [2]:
import timm
from torch import nn

pretrained_model = TAMELIT.load_from_checkpoint(
    "/home/ntrougkas/Documents/3D-TAME/T-TAME-small/vkep09vs/checkpoints/epoch=7-step=320296.ckpt",
    model=timm.create_model("deit_small_patch16_224", pretrained=True),
)
pretrained_model.generic.attn_mech[1].fuser = nn.Sequential(
    pretrained_model.generic.attn_mech[1].fuser,
    nn.Conv2d(
        in_channels=1000,
        out_channels=num_classes,
        kernel_size=1,
        padding=0,
        bias=True,
    ),
)
pl_model.generic.attn_mech[1] = pretrained_model.generic.attn_mech[1]

INFO:timm.models.helpers:Loading pretrained weights from url (https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth)


Train

In [3]:
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import LearningRateMonitor

import torch

torch.set_float32_matmul_precision("medium")
wandb_logger = WandbLogger(project="APTOS2019-TAME", name="pretrained_case3")
trainer = pl.Trainer(
    enable_checkpointing=True,
    max_epochs=epochs,
    logger=wandb_logger,
    log_every_n_steps=1,
    callbacks=[
        LearningRateMonitor(
            logging_interval="step", log_momentum=True, log_weight_decay=True
        )
    ],
)
trainer.fit(pl_model, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marios1861. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name           | Type               | Params
------------------------------------------------------
0 | generic        | Generic            | 23.3 M
1 | train_accuracy | MulticlassAccuracy | 0     
2 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
1.6 M     Trainable params
21.7 M    Non-trainable params
23.3 M    Total params
93.086    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.


Test

In [4]:
_, test_loader, _ = mt.get_jupyter_items(train=False)
trainer.test(pl_model, test_loader)

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

[{}]

In [5]:
wandb_logger.finalize("success")
wandb_logger.experiment.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LR/pg1,▁▂▂▃▃▄▅▆▇▇███████▇▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁
LR/pg1-momentum,██▇▇▆▅▄▃▂▂▁▁▁▁▁▁▁▂▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇▇█████
LR/pg1-weight_decay,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
LR/pg2,▁▂▂▃▃▄▅▆▇▇███████▇▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁
LR/pg2-momentum,██▇▇▆▅▄▃▂▂▁▁▁▁▁▁▁▂▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇▇█████
LR/pg2-weight_decay,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/Accuracy,█▂▄▄▂▄▂▅▄▃▄█▃▅▆▅▆▃▅▆▄▇▃▃▄▃▁▅▃▃▅█▂▆▃▅▃▄▆▄
training/CE,▁██████▇███▇█▇██▇█▇██▇█████████▇█▇████▇█
training/Loss,█▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


# Calculate AD and IC for competing methods

## First apply on gradcam

In [36]:
import classification as mt

net, test_loader, _ = mt.get_jupyter_items(train=False)
net.eval()
from tame.utilities.comp_module import CompareModel
import torch
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger

model = net

pl_model = CompareModel(
    mdl_name="vit",
    name="gradcam",
    raw_model=model,
    target_layers=[model.backbone.blocks[-1]],
    eval_length="long",
    print_adic=True,
)


wandb_logger = WandbLogger(project="APTOS2019-TAME", name="gradcam")
torch.set_float32_matmul_precision("medium")
trainer = pl.Trainer(logger=wandb_logger)

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


In [2]:
%load_ext autoreload
%autoreload 2

In [37]:
trainer.test(pl_model, test_loader)

/home/ntrougkas/.cache/pypoetry/virtualenvs/tame-cJ7vbEx--py3.8/lib/python3.8/site-packages/lightning/pytorch/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

1
2


[{}]

In [4]:
wandb_logger.finalize("success")
wandb_logger.experiment.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


## Do the same for the rest of the methods

In [39]:
from tame.utilities.comp_module import CompareModel
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
import classification as mt
import torch

methods = [
    # "scorecam",
    "gradcam++",
]

net, test_loader, _ = mt.get_jupyter_items(train=False)
net.eval()
model = net
print(test_loader)
for method_name in methods:
    pl.utilities.memory.garbage_collection_cuda()
    torch.cuda.empty_cache()
    pl_model = CompareModel(
        mdl_name="vit",
        name=method_name,
        raw_model=model,
        target_layers=[model.backbone.blocks[-1].norm1],
        eval_length="short",
    )
    if method_name == "scorecam":
        pl_model.cam_model.batch_size = 4
    wandb_logger = WandbLogger(project="APTOS2019-TAME", name=method_name)
    torch.set_float32_matmul_precision("medium")
    trainer = pl.Trainer(logger=wandb_logger)
    trainer.test(pl_model, test_loader)
    wandb_logger.finalize("success")
    wandb_logger.experiment.finish()

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

1
2


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Generate examples for TAME method

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import os
from pathlib import Path

import torch
from torchvision import models
import classification as mt

from tame.utilities.pl_module import TAMELIT
net, test_loader, _ = mt.get_jupyter_items(train=False)
net.cpu()
model = TAMELIT.load_from_checkpoint(
    # "/home/ntrougkas/Documents/3D-TAME/medical_transformers/APTOS2019-TAME/6i01rwf8/checkpoints/epoch=7-step=320.ckpt",
    "/home/ntrougkas/Documents/3D-TAME/medical_transformers/APTOS2019-TAME/d1el6nks/checkpoints/epoch=7-step=320.ckpt",
    model=net
)
# resnet50
# model_name = "resnet50"
# model = TAMELIT.load_from_checkpoint("logs/TAME_resnet50/version_5/checkpoints/epoch=3-step=160148.ckpt", train_method="legacy")

# model: pl.LightningModule = torch.compile(model)  # type: ignore
classes = [
    "No DR",
    "Mild DR",
    "Moderate DR",
    "Severe DR",
    "Prolific DR",
]
images, labels = next(iter(test_loader))
for id in [5]:
    model.save_masked_image(
        images[id],
        id,
        "APTOS19-ViT",
        labels[id],
        classes[labels[id]],
        classes,
        # select_mask=130
    )

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino
0


Generate examples for other methods

In [16]:
import os
from pathlib import Path

import torch
from torchvision import models
import classification as mt

from tame.utilities.comp_module import CompareModel

net, test_loader, _ = mt.get_jupyter_items(train=False)
net.eval()

net.cpu()
model = net

methods = [
    "gradcam",
    "scorecam",
    "gradcam++",
]

images, labels = next(iter(test_loader))
for method_name in methods:
    pl_model = CompareModel(
        mdl_name="vit",
        name=method_name,
        raw_model=model,
        target_layers=[model.backbone.blocks[-1].norm1],
        eval_length="long",
    )
    mdl_truth = model(images[id].unsqueeze(0).cuda()).argmax().item()
    classes = [
        "No DR",
        "Mild DR",
        "Moderate DR",
        "Severe DR",
        "Prolific DR",
    ]
    for id in [5]:
        pl_model.save_masked_image(
            images[id],
            id,
            labels[id],
            classes[labels[id]],
            mdl_truth,
            classes[mdl_truth]
        )

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino
torch.Size([1, 1, 224, 224])


100%|██████████| 48/48 [00:00<00:00, 125.87it/s]


torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
